In [1]:
from ultralytics.utils.custom_utils.init_setup import setup, delete_all_fiftyone_datasets

delete_all_fiftyone_datasets()
setup(0)

Loading dataset
 100% |███████████████| 7359/7359 [15.0s elapsed, 0s remaining, 401.8 samples/s]      
 100% |███████████████| 3977/3977 [10.1s elapsed, 0s remaining, 353.0 samples/s]      
 100% |███████████████| 2468/2468 [6.7s elapsed, 0s remaining, 356.4 samples/s]      
Adding transects


13804it [00:26, 526.08it/s]


Adding dataset attributes


100%|██████████| 13804/13804 [01:31<00:00, 151.48it/s]


In [2]:
import fiftyone as fo
from tqdm import tqdm
from ultralytics.utils.custom_utils.helpers import get_fiftyone_dataset

train = [
    "transect12",
    "2022-09-03",
    "transect02",
    "2022-09-04",
    "Transect 2",
    "Transect 6",
    "Transect 7",
    "Transect 13",
    "Transect 3",
    "Transect 11",
    "Transect 16",
    "Transect 17",
    "Transect 10",
    "Transect 4",
    "Transect 8",
    "Transect 14",
    "transect05",
]

val = [ 
    "transect15",
    "Transect 18",
    "Transect 15",
    "Transect 4",
    "Transect 1",
]

test = [ 
    "Transect 5",
    "2022-08-31",
    "Transect 12",
    "transect03",
    "transect04",
    "transect09",
    "new",
]

dataset, classes = get_fiftyone_dataset(0)

for sample in tqdm(dataset):
    if sample.transect in train:
        sample["split"] = "train"
    elif sample.transect in val:
        sample["split"] = "val"
    else:
        sample["split"] = "test"
    sample.save()

sesh = fo.launch_app(dataset)



Loading dataset


100%|██████████| 13760/13760 [00:28<00:00, 489.81it/s]


In [4]:
dataset

Dataset:     My_dataset_new0
Media type:  image
Num samples: 13760
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    transect:   fiftyone.core.fields.StringField
    split:      fiftyone.core.fields.StringField
View stages:
    1. Match(filter={'$expr': {'$gt': [...]}})